In [22]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import cv2
from sklearn.model_selection import train_test_split
import keras
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Dense, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization


In [23]:
df_train = pd.read_csv('/Users/umid/Downloads/Dog/labels.csv')
df_test = pd.read_csv('/Users/umid/Downloads/Dog/sample_submission.csv')
targets_series = pd.Series(df_train['breed'])

In [24]:
one_hot = pd.get_dummies(targets_series, sparse=True)
one_hot_labels = np.asarray(one_hot)

# Image size to resize dataset
im_size = 160

i = 0
x_train = []
y_train = []
x_test = []

In [25]:
#  load the image, resize and append to the train dataset
for f, breed in tqdm(df_train.values):
    img = cv2.imread('/Users/umid/Downloads/Dog/train/{}.jpg'.format(f))
    label = one_hot_labels[i]
    x_train.append(cv2.resize(img, (im_size, im_size)))
    y_train.append(label)
    i += 1


100%|██████████| 10222/10222 [00:43<00:00, 236.37it/s]


In [26]:
# Convert values
y_train_raw = np.array(y_train, np.uint8)
x_train_raw = np.array(x_train, np.float32)


In [27]:
# Data normalization
mean = x_train_raw.mean()
std = x_train_raw.std()
x_train_raw = (x_train_raw - mean) / std


In [28]:
# Printing the shape
print(x_train_raw.shape)
print(y_train_raw.shape)

num_class = y_train_raw.shape[1]

(10222, 160, 160, 3)
(10222, 120)


In [30]:
# Split
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=0.25, random_state=1)


In [31]:
# Create the base pre-trained model
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(im_size, im_size, 3))


In [32]:
# Add more layers
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='sigmoid')(x)
x = BatchNormalization()(x)
x = Dense(1024, activation='sigmoid')(x)
x = BatchNormalization()(x)
x = Dense(256, activation='sigmoid')(x)
x = BatchNormalization()(x)
x = Dense(32, activation='sigmoid')(x)
prediction_layer = Dense(num_class, activation='softmax')(x)



In [33]:
# This is the model we will train
model = Model(inputs=base_model.input, outputs=prediction_layer)


In [34]:
# train
for layer in base_model.layers:
    layer.trainable = False

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [35]:
# Callback to stop if val_acc stop improving
callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 160, 160, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 160, 160, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 160, 160, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 80, 80, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 80, 80, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 80, 80, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 40, 40, 128)       0         
__________

In [14]:
# Fitting the model. I am using my own computer so it is taking to much time so I decided to use lower epoch. 
model.fit(X_train, Y_train, epochs=1, validation_data=(X_valid, Y_valid), verbose=1, callbacks=callbacks_list)

Train on 7155 samples, validate on 3067 samples
Epoch 1/1
7155/7155 [==============================] - 1756s 245ms/step - loss: 4.4947 - acc: 0.0481 - val_loss: 3.8320 - val_acc: 0.1190


In [36]:
# Final evaluation of the model
#scores = model.evaluate(X_train, Y_train, verbose=0)
#print("CNN Error: %.2f%%" % (100-scores[1]*100))



In [37]:
# Load the data for the test dataset
for f in tqdm(df_test['id'].values):
    img = cv2.imread('/Users/umid/Downloads/Dog/test/{}.jpg'.format(f))
    x_test.append(cv2.resize(img, (im_size, im_size)))

x_test = np.array(x_test, np.float32)
x_test = (x_test - mean) / std


100%|██████████| 10357/10357 [00:42<00:00, 241.15it/s]


In [38]:
# Making predictions about the test data
predictions = model.predict(x_test, verbose=1)

sub = pd.DataFrame(predictions)



10357/10357 [==============================] - 3129s 302ms/step


In [39]:
# Set column names 
col_names = one_hot.columns.values
sub.columns = col_names

In [40]:
# Insert the column id from the sample_submission
sub.insert(0, 'id', df_test['id'])


In [41]:
# Save the data
sub.to_csv('submission.csv', index=False)
